## Test the model with videos and Images

In [1]:
import tensorflow as tf
tf.__version__

'2.1.2'

In [2]:
import cv2
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

In [3]:
import numpy as np
from tensorflow.keras.models import load_model

In [4]:
model_path = r'./face_mask_model.h5'
mask_model = load_model(model_path)

In [5]:
face_cascade = cv2.CascadeClassifier()

In [6]:
if not face_cascade.load(cv2.samples.findFile(r'./haarcascade_frontalface_default.xml')):
    print("Error loading in file")
    exit(0)

In [7]:
labels_dict = {0: 'with mask', 1:'Without mask'}
color_dict = {0:(0,255,0), 1:(0,0,255)}

In [22]:
# Cpturing the webcam
source = cv2.VideoCapture(0)

In [23]:
while(True):

    ret,img=source.read()
    gray_img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    face_dim=face_cascade.detectMultiScale(gray_img, 1.1, 5)  

    for (x,y,w,h) in face_dim:
    
        face_img=gray_img[y:y+w,x:x+w]
        resized=cv2.resize(face_img,(128,128))
        resized = resized/255.0
        reshaped = np.reshape(resized, (1, 128, 128, 1))
        result=mask_model.predict(reshaped)
        confidence = str(np.around(100 * np.max(result[0]), 2))
        label=np.argmax(result,axis=1)[0]
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(img, labels_dict[label]+'-'+confidence, (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
        
    cv2.imshow('LIVE',img)
    
    if cv2.waitKey(1) & 0XFF == ord('p'):  # Press 'p' key for exit
        break
        
cv2.destroyAllWindows()
source.release()

# Test with Images

In [41]:
image_path = 'Path of your image'
image = cv2.imread(image_path)

In [42]:
image.shape

(1235, 916, 3)

In [43]:
face = face_cascade.detectMultiScale(image, 1.3, 5) 
for (x, y, w, h) in face:
    face_img=image[y:y+w,x:x+w]
    face_img=cv2.cvtColor(face_img,cv2.COLOR_BGR2GRAY)
    resized=cv2.resize(face_img,(128,128))
    reshaped = np.reshape(resized, (1, 128, 128, 1))
    result=mask_model.predict(normalized)
    confidence = str(np.around(100 * np.max(result[0]), 2))
    label=np.argmax(result,axis=1)[0]
    cv2.rectangle(image,(x,y),(x+w,y+h),color_dict[label],2)
    cv2.rectangle(image,(x,y-40),(x+w,y),color_dict[label],-1)
    cv2.putText(image, labels_dict[label]+'-'+confidence, (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)   
        
cv2.imshow('Image',image)
    
cv2.waitKey(5000)
        
cv2.destroyAllWindows()